In [2]:
import sys
# sys.path.append(r'/home/wyf/0code')
sys.path.append(r'E:/0code')

In [3]:
import pyml
import pandas as pd
import numpy as np
from pyml.neighbors.regression import KNeighborsRegressor
from pyml.preprocessing import z_score

# 读取数据文件

In [4]:
train = pd.read_excel('./data/train.xlsx')
test = pd.read_excel('./data/testStudent.xlsx')

In [5]:
train.dtypes # 检查有没有数据类型错误的，比如原本是int的变成str，说明里面可能有nan值等奇怪的数据

Additional_Number_of_Scoring                    int64
Average_Score                                 float64
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Tags                                           object
Reviewer_Score                                float64
dtype: object

In [6]:
train.Tags[0].strip("[]").replace("'",'').split(',')

[' Family with young children ', '  Standard Queen Room ', '  Stayed 1 night ']

In [7]:
def preprocessing_data(df):
    pass

# 构建训练集与测试集

In [8]:
train_label = train['Reviewer_Score']
train_feat = train.drop('Reviewer_Score', axis=1)
test_feat = test

# 定义特征构建函数

https://github.com/ShawnyXiao/2017-CCF-BDCI-Enterprise 该师兄的分析我觉得蛮有道理的


1. 基础特征
2. 偏离值特征
3. 交叉特征
4. 想象力特征hhh

观察数据，发现
1. 通过比值构建新特征：构造积极、消极、评论员评论占比特征

In [10]:
def get_proportion_feature(df):
    df = df.copy()
    
    base_features = ['Total_Number_of_Reviews']
    gap_features = ['Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given']
    for base_feature in base_features:
        for gap_feature in gap_features:
            df[gap_feature+'_radio_'+base_feature] = df[gap_feature]/df[base_feature] * 10
            # 数字太小了，乘上一个10
            df = df.drop(gap_feature, axis=1)
    return df

In [11]:
def normalized_df(df):
    # 将数据归一化
    return (df - df.min()) / (df.max() - df.min())

# 调用函数 构建特征

In [12]:
# 暂时不处理tags数据，就将其丢弃
train_feat = train_feat.drop(labels=['Tags'], axis=1)
test_feat = test_feat.drop(labels=['Tags'], axis=1)

In [13]:
train_feat = get_proportion_feature(train_feat)
test_feat = get_proportion_feature(test_feat)

In [14]:
train_feat = normalized_df(train_feat)
test_feat = normalized_df(test_feat)

# 导入模型与训练

In [20]:
kreg = KNeighborsRegressor()

In [21]:
kreg.fit(train_feat.values, train_label.values.reshape(-1,1))

In [ ]:
y_pred = kreg.predict(test_feat.values, watch=True)

[8.08190717]
[8.09952379]
[8.35757885]
[9.43691947]
[7.41920796]
[7.33975009]
[6.3501725]
[8.4949593]
[8.76331567]
[7.52586242]
current status: 10/30000
[9.05845951]
[7.51877912]
[8.86847326]
[7.02870151]
[7.58043562]
[9.59877051]
[7.52685215]
[9.52073691]
[7.33953568]
[7.16006795]
current status: 20/30000
[8.50892888]
[7.2595419]
[8.76151751]
[9.25990055]
[7.31097397]
[8.5194791]
[9.50226637]
[8.59963714]
[9.10144231]
[7.50209619]
current status: 30/30000
[9.02092371]
[8.59789441]
[7.93972279]
[6.43025767]
[8.08145068]
[9.33109512]
[8.93939431]
[8.48267988]
[9.0031418]
[8.41554493]
current status: 40/30000
[5.3477349]
[6.52162438]
[7.32335839]
[9.26026842]
[3.86092044]
[6.08409463]
[9.59996216]
[8.77622275]
[9.29172089]
[8.31749509]
current status: 50/30000
[7.00111618]
[8.92090378]
[9.6799782]
[5.89991577]
[8.50249428]
[8.85306728]
[9.10001844]
[8.18128715]


# 结果文件的写出

In [ ]:
sub = pd.DataFrame(y_pred)
sub.to_csv('./results/'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)